In [2]:
import cv2
video = cv2.VideoCapture(0)

haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


while True:
    ret, frame = video.read()
    # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    faces_rect = haar_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=9)

    if faces_rect != []:
        for (x,y,w,h) in faces_rect:
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 2)

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows() 
            


C:\Users\ACER\AppData\Local\Temp\ipykernel_9204\1212474689.py:13: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if faces_rect != []:


In [1]:
import cv2
from mtcnn.mtcnn import MTCNN

detector = MTCNN()
video = cv2.VideoCapture(0)     # dùng webcam, nếu có video thì truyền đường dẫn vào

def draw_faces_and_extract(frame, faces):
    face_images = []
    for face in faces:
        x, y, width, height = face['box']
        data = frame[y:y+height, x:x+width]
        cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 155, 255), 2)

        for key, value in face['keypoints'].items():
            cv2.circle(frame, value, 2, (0, 155, 255), 2)

        face_images.append(data)
    
    return frame, face_images
    
while True:
    # lấy frame
    ret, frame = video.read()
    # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    faces = detector.detect_faces(frame)
    
    if faces:
        frame, face_images = draw_faces_and_extract(frame, faces)

    # # hiển thị kết quả
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):   # nhấn q nếu muốn thoát
        break

video.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 31ms/step


In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import jaccard_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Đọc tập dữ liệu từ file CSV
# Giả sử tập dữ liệu có cột 'xmin', 'ymin', 'xmax', 'ymax' đại diện cho bounding boxes
df = pd.read_csv('du_lieu_bounding_boxes.csv')

# Tính toán chiều rộng và chiều cao từ bounding boxes
df['width'] = df['xmax'] - df['xmin']
df['height'] = df['ymax'] - df['ymin']

# Tạo mảng 'sizes' từ 'width' và 'height'
sizes = df[['width', 'height']].values

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
sizes = scaler.fit_transform(sizes)

# Khởi tạo danh sách để lưu trữ giá trị IoU
iou_values = []

# Khởi tạo danh sách số lượng anchor boxes
num_anchors_list = list(range(1, 16))

# Vòng lặp tối ưu cả số lượng và kích thước
for num_anchors in num_anchors_list:
    kmeans = KMeans(n_clusters=num_anchors, random_state=42)
    kmeans.fit(sizes)
    labels_true = kmeans.predict(sizes)
    anchor_sizes = kmeans.cluster_centers_

    labels_pred = kmeans.predict(anchor_sizes)
    
    mean_iou = jaccard_score(labels_true, labels_pred, average='macro')

    # Thêm giá trị IoU vào danh sách
    iou_values.append(mean_iou)

# Tìm số lượng anchor boxes tốt nhất
best_num_anchors = num_anchors_list[np.argmax(iou_values)]
best_mean_iou = iou_values[np.argmax(iou_values)]
best_anchor_sizes = KMeans(n_clusters=best_num_anchors, random_state=42).fit(sizes).cluster_centers_

# In ra số lượng anchor boxes và kích thước tốt nhất
print(f"Best Number of Anchors: {best_num_anchors}")
print(f"Best Anchor Sizes:\n{best_anchor_sizes}")

# Vẽ biểu đồ số lượng anchor boxes và giá trị trung bình của IoU
plt.plot(num_anchors_list, iou_values, '-o')
plt.scatter(best_num_anchors, best_mean_iou, c='red', marker='o', label='Best Number of Anchors')
plt.ylabel("Mean IoU")
plt.xlabel("Number of Anchors")
plt.title("Number of Anchors vs. Mean IoU")
plt.legend()
plt.show()
